In [ ]:
from typing import Union, Tuple, List
from transformers import BertConfig, BertForPreTraining, BertTokenizerFast
import numpy as np
import random
import pandas as pd
from typing import Union, Tuple, List
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
import numpy as np
import random
import pandas as pd
from datetime import datetime, date
from tqdm.notebook import tqdm
from sklearn.metrics import log_loss, accuracy_score, precision_score, recall_score
from tqdm import tqdm
# from IPython.display import Image
from joblib import Parallel, delayed
import torch
import torch.nn as nn
from torch.nn.init import normal_
from torch.utils.data import TensorDataset, DataLoader
import torch
from transformers import BertTokenizer, BertModel
import re
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)
from torch.utils.data.dataset import Dataset
from transformers import DataCollatorForLanguageModeling
import os
from transformers import Trainer, TrainingArguments, AutoTokenizer
from transformers.utils import logging
from transformers import BertConfig, BertForPreTraining, BertTokenizerFast
from filelock import FileLock
import time
import pickle
from transformers import BertConfig, BertForPreTraining
from tokenizers import BertWordPieceTokenizer
import argparse

from transformers import Trainer, TrainingArguments, AutoTokenizer
from datetime import datetime, date
from tqdm.notebook import tqdm
from sklearn.metrics import log_loss, accuracy_score, precision_score, recall_score
from tqdm import tqdm
# from IPython.display import Image
from joblib import Parallel, delayed
import torch
import torch.nn as nn
from torch.nn.init import normal_
from torch.utils.data import TensorDataset, DataLoader
import torch
from transformers import BertTokenizer, BertModel
import re
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)
from torch.utils.data.dataset import Dataset
import matplotlib.pyplot as plt
from transformers import DataCollatorForLanguageModeling
import os
from transformers import Trainer, TrainingArguments
from transformers.utils import logging
logger = logging.get_logger(__name__)
from filelock import FileLock
import time
import pickle

In [ ]:
wine_df = pd.read_csv('/opt/ml/wine/data/wine_df.csv')
review_df = pd.read_csv('/opt/ml/wine/data/review_df_total.csv',encoding = 'utf-8-sig').loc[:,['user_url','rating','text','wine_url']]

In [ ]:
import json
with open('/opt/ml/wine/code/data/feature_map/item2idx.json','r') as f:
    item2idx = json.load(f)

In [ ]:
review_df = review_df[review_df['text'].isna()==False]

In [ ]:
review_df['text'] = review_df['text'].apply(lambda x: x + '.' if x[-1] != '.' else x)

In [ ]:
def keep_english_and_digits(text):
    # Remove any characters that are not English alphabets, digits, periods, or commas at the end of sentences
    clean_text = re.sub(r'[^a-zA-Z0-9\s.,]', '', text)
    return clean_text

In [ ]:
review_df['text'] = review_df['text'].apply(keep_english_and_digits)

In [ ]:
review_df['wine_id'] = review_df['wine_url'].map(item2idx)
review_df = review_df[review_df['wine_id'].isna()==False]
review_df['wine_id'] = review_df['wine_id'].astype('int').astype('category')

In [ ]:
note_df = wine_df.filter(like='_child')

In [ ]:
notes = {}
import ast
def str2dict(x):
    try: return ast.literal_eval(x)
    except: return {}
def get_keys(x):
    return set(x.keys())

for col in note_df.columns:
    note_df.loc[:,col] = note_df.loc[:,col].apply(str2dict)
    sub_note = set()
    for i in tqdm(range(len(note_df))):
        subs = get_keys(note_df[col][i])
        sub_note = sub_note | subs
    notes[col.replace('_child','')] = sub_note

In [ ]:
text_with_note = review_df.drop(['rating','wine_id'], axis = 1)

# 와인 기본 정보 multi label

In [ ]:
wine_basic_info = wine_df.loc[:, ['url','country','region','grape','winetype']]
wine_basic_info['wine_id'] = wine_basic_info['url'].map(item2idx)
wine_basic_info= wine_basic_info[wine_basic_info['wine_id'].isna()==False]
wine_basic_info['wine_id'] = wine_basic_info['wine_id'].astype('int').astype('category')

In [126]:
def check_note_in_review(text, notes_data):
    text = text.lower()
    result = []
    for key in notes:
        if any(word in text for word in notes[key]):
            result.append(1)
        else: result.append(0)
    return result

def marking_data(df, notes_data):
    df.reset_index(inplace = True)
    data = []
    for i in tqdm(range(len(df))):
        note_onehot = check_note_in_review(df.loc[i,'text'], notes_data)
        tmp = {}
        tmp['text'] = df.loc[i,'text']
        tmp['label'] = note_onehot
        data.append(tmp)
        
    return pd.DataFrame(data)


def parallel_dataframe_2input(func, df, notes_data, num_cpu):

    for key in notes_data: notes_data[key] = set(notes_data[key])
    chunks = np.array_split(df, num_cpu)

    print('Parallelizing with ' +str(num_cpu)+'cores')
    with Parallel(n_jobs = num_cpu, backend="multiprocessing") as parallel:
        results = parallel(delayed(func)(chunks[i], notes_data) for i in range(num_cpu))

    for i,data in enumerate(results):
        if i == 0:
            output = data
        else:
            output = pd.concat([output, data], axis=0)
    output.reset_index(inplace = True)
    return output

In [118]:
text = pd.DataFrame(text_with_note.loc[:,'text'])

In [119]:
text_with_notelabel = parallel_dataframe_2input(marking_data, text, notes, 8)

Parallelizing with 8cores


100%|██████████| 928506/928506 [03:07<00:00, 4946.01it/s]

[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]

In [138]:
text_with_notelabel.to_csv('/opt/ml/wine/data/text_with_notelabel.csv', index = False, encoding = 'utf-8')

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe.text
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            truncation = True,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
print('학습 문장 최대 길이 :',max(len(l) for l in text_with_notelabel['text']))
print('학습 문장의 평균 길이 :',sum(map(len, text_with_notelabel['text']))/len(text_with_notelabel['text']))

plt.hist([len(s) for s in text_with_notelabel['text']], bins=50)
plt.xlabel('length of data')
plt.ylabel('number of data')
plt.show()

In [ ]:
# Creating the dataset and dataloader for the neural network
train_size = 0.8
train_dataset = text_with_notelabel.sample(frac=train_size,random_state=200)
test_dataset = text_with_notelabel.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


In [ ]:

print("FULL Dataset: {}".format(text_with_notelabel.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

training_set = CustomDataset(train_dataset, tokenizer, 156)
testing_set = CustomDataset(test_dataset, tokenizer, 156)

In [129]:
TRAIN_BATCH_SIZE = 512
VALID_BATCH_SIZE = 1024
EPOCHS = 50
LEARNING_RATE = 1e-05

In [130]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [131]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self, num_labels):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, num_labels)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
model = BERTClass(len(text_with_notelabel['label'][0]))
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [132]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [133]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [136]:
def train(epoch):
    model.train()
    for data in tqdm(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [137]:
for epoch in range(EPOCHS):
    train(epoch)
    

  0%|          | 0/11607 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 192.00 MiB (GPU 0; 31.75 GiB total capacity; 26.58 GiB already allocated; 157.75 MiB free; 26.88 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
for epoch in range(EPOCHS):
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")